In [ ]:
import os
import glob
import pandas as pd
import numpy as np

#set working directory to this from the workspace
os.chdir('/Users/janjarco/Programming/PrivateRepository/FlightDataCustomerSegmentation')

#list all the files in current working directory
print(os.listdir("data/clicks"))


['meta_clicks.split-171.json', 'meta_clicks.split-54.json', 'meta_clicks.split-126.json', 'meta_clicks.split-219.json', 'meta_clicks.split-42.json', 'meta_clicks.split-130.json', 'meta_clicks.split-188.json', 'meta_clicks.split-15.json', 'meta_clicks.split-167.json', 'meta_clicks.split-235.json', 'meta_clicks.split-81.json', 'meta_clicks.split-39.json', 'meta_clicks.split-223.json', 'meta_clicks.split-97.json', 'meta_clicks.split-78.json', 'meta_clicks.split-192.json', 'meta_clicks.split-58.json', 'meta_clicks.split-2.json', 'meta_clicks.split-203.json', 'meta_clicks.split-215.json', 'meta_clicks.split-19.json', 'meta_clicks.split-184.json', 'meta_clicks.split-110.json', 'meta_clicks.split-62.json', 'meta_clicks.split-147.json', 'meta_clicks.split-35.json', 'meta_clicks.split-151.json', 'meta_clicks.split-23.json', 'meta_clicks.split-106.json', 'meta_clicks.split-74.json', 'meta_clicks.split-107.json', 'meta_clicks.split-75.json', 'meta_clicks.split-150.json', 'meta_clicks.split-22.jso

In [3]:
def list_files_from_path(path):
    # pattern: pattern to match the json files
    # path: path to the directory containing the json files
    # Load all json files in the directory givent by path that match the given pattern
    file_names = []
    for file_name in os.listdir(path):
        if file_name.endswith(".json"):
            file_names.append(path+"/"+file_name)
    return file_names
list_files_from_path("data/clicks")

['data/clicks/meta_clicks.split-171.json',
 'data/clicks/meta_clicks.split-54.json',
 'data/clicks/meta_clicks.split-126.json',
 'data/clicks/meta_clicks.split-219.json',
 'data/clicks/meta_clicks.split-42.json',
 'data/clicks/meta_clicks.split-130.json',
 'data/clicks/meta_clicks.split-188.json',
 'data/clicks/meta_clicks.split-15.json',
 'data/clicks/meta_clicks.split-167.json',
 'data/clicks/meta_clicks.split-235.json',
 'data/clicks/meta_clicks.split-81.json',
 'data/clicks/meta_clicks.split-39.json',
 'data/clicks/meta_clicks.split-223.json',
 'data/clicks/meta_clicks.split-97.json',
 'data/clicks/meta_clicks.split-78.json',
 'data/clicks/meta_clicks.split-192.json',
 'data/clicks/meta_clicks.split-58.json',
 'data/clicks/meta_clicks.split-2.json',
 'data/clicks/meta_clicks.split-203.json',
 'data/clicks/meta_clicks.split-215.json',
 'data/clicks/meta_clicks.split-19.json',
 'data/clicks/meta_clicks.split-184.json',
 'data/clicks/meta_clicks.split-110.json',
 'data/clicks/meta_cli

In [59]:
from collections import OrderedDict
import json

def flatten_nested_json_df(df):
    df = df.reset_index()
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()
    
    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()

    
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []

        for col in dict_columns:
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace

        for col in list_columns:
            #print(f"exploding: {col}")
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)

        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()
    return df

def load_data(source_path, table_name):
    # Load file names from source path
    file_names = list_files_from_path(source_path)
    
    # Initialize empty list to hold data frames
    data_frames = []
    
    # Iterate over file names
    for file_name in file_names:

        # Read JSON file
        nested_df = pd.read_json(file_name, lines=True)
        
        # Flatten nested JSON
        df_flatten = flatten_nested_json_df(nested_df)
        
        # Append flattened data frame to list of data frames
        data_frames.append(df_flatten)
    
    # Concatenate all data frames from list
    df_merged = pd.concat(data_frames, ignore_index=True)
    
    # Rename all columns with prefix based on table_name
    df_merged = df_merged.rename(columns=lambda x: f"{table_name}_{x}")
    
    # Rename all columns with prefix based on table_name
    df_merged.columns = df_merged.columns.str.replace("__", "_")
    df_merged.columns = df_merged.columns.str.replace("source.", "")
    
    # Sort data frame by search_id
    df_merged = df_merged.sort_values(by=f"{table_name}_search_id")
    
    # Remove duplicate search_id's
    df_merged = df_merged.drop_duplicates(subset=f"{table_name}_search_id", keep='first')

    return df_merged



In [60]:
orders = load_data("data/orders", "orders")
orders.columns.values

/var/folders/gp/cz1c1p2d6ln8dw3vyy038cwm0000gn/T/ipykernel_72289/3789868186.py:61: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merged.columns = df_merged.columns.str.replace("source.", "")


array(['orders_index', 'orders_index', 'orders_id', 'orders_score',
       'orders_order_id', 'orders_search_id', 'orders_gate_id',
       'orders_order_type', 'orders_user_ipaddress', 'orders_paid',
       'orders_paid_at', 'orders_paid_at_date', 'orders_paid_transact',
       'orders_cancelled', 'orders_cancelled_at',
       'orders_cancelled_description', 'orders_created_at',
       'orders_created_at_date', 'orders_midoffice_id',
       'orders_paid_unique_id', 'orders_searchrule_action_id',
       'orders_type', 'orders_timestamp', 'orders_timestamp_date',
       'orders_logreader_type', 'orders_search_data.search_id',
       'orders_search_data.gate_id', 'orders_search_data.gate_name',
       'orders_search_data.meta_search', 'orders_search_data.meta_clicks',
       'orders_search_data.status', 'orders_search_data.process_status',
       'orders_search_data.search_from_ip',
       'orders_search_data.search_type',
       'orders_search_data.search_parameters.type',
       'orders

In [61]:
orders.head(n=20)

,orders_index,orders_index,orders_id,orders_score,orders_order_id,orders_search_id,orders_gate_id,orders_order_type,orders_user_ipaddress,orders_paid,...,orders_search_data.experiments.202204-danish-explanations,orders_search_data.search_parameters.only_public_fares,orders_search_data.experiments.202206-sabre-gateway-migration,orders_midoffice_contact_id,orders_midoffice_quotation_id,orders_search_data.experiments.202202-test-experiment,orders_search_data.experiments.202202-handbag-ordering,orders_search_data.experiments.202203-german-background-image,orders_search_data.experiments.202203-mix-classic,orders_search_data.experiments.202205-meta-per-pax
73593,827,orders,977450,1,977450,3157728355,82,addon,None,True,...,hypothesis,NaN,NaN,667583.0,4066485.0,NaN,NaN,NaN,NaN,NaN
65530,5877,orders,935524,1,935524,3165582470,108,addon,None,False,...,NaN,NaN,NaN,668455.0,NaN,control,NaN,NaN,NaN,NaN
138299,5764,orders,983054,1,983054,3220257758,13,addon,None,False,...,NaN,NaN,NaN,675421.0,NaN,NaN,NaN,NaN,NaN,NaN
134635,2100,orders,940931,1,940931,3223539981,6,addon,None,False,...,NaN,NaN,NaN,675835.0,NaN,hypothesis,hypothesis,NaN,NaN,NaN
173028,1070,orders,930893,1,930893,3245961443,37,addon,None,True,...,NaN,NaN,NaN,678846.0,4076720.0,hypothesis,NaN,NaN,NaN,NaN
87999,1955,orders,915425,1,915425,3273655538,13,addon,None,False,...,NaN,NaN,NaN,682443.0,NaN,NaN,NaN,NaN,NaN,NaN
91106,5062,orders,951857,1,951857,3312861152,126,addon,None,False,...,NaN,NaN,NaN,688242.0,NaN,control,NaN,control,NaN,NaN
100163,885,orders,1207631,1,1207631,3327266231,82,addon,None,True,...,hypothesis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174839,2881,orders,952072,1,952072,3331758385,126,addon,None,True,...,NaN,NaN,NaN,690777.0,4101043.0,hypothesis,NaN,control,NaN,NaN
157220,4943,orders,1001875,1,1001875,3336730708,37,addon,None,False,...,NaN,NaN,NaN,691488.0,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
clicks = load_data("data/clicks", "clicks")
clicks.head(n = 20)


/var/folders/gp/cz1c1p2d6ln8dw3vyy038cwm0000gn/T/ipykernel_72289/3789868186.py:61: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merged.columns = df_merged.columns.str.replace("source.", "")
/Users/janjarco/miniconda3/envs/tensorflow/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/Users/janjarco/miniconda3/envs/tensorflow/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,clicks_index,clicks_index,clicks_id,clicks_score,clicks_id,clicks_created_at,clicks_search_id,clicks_search_created_at,clicks_gate_id,clicks_meta_ref,...,clicks_itinerary_direct_flight,clicks_itinerary_travel_time,clicks_itinerary_with_baggage,clicks_result_set_position,clicks_result_set_position_sales_price,clicks_result_set_cheapest_price,clicks_result_set_fastest_price,clicks_result_set_best_price,clicks_meta_loopback_price,clicks_meta_clicked_price
1943883,4,meta_clicked_itineraries,kXtIE34BCwb6y2g4GG-y,1,5009867,2022-01-01T02:36:08.000+01:00,3575609108,2022-01-01T02:13:57.000+01:00,37,BDf9_eGU1KX3T7PZ9BU9bQ,...,True,445.0,False,0.0,0.0,5114.0,5114.0,5114.0,NaN,NaN
1943880,1,meta_clicked_itineraries,Lmk2E34B_h5ol0SVvXEE,1,5009860,2022-01-01T02:17:10.000+01:00,3575610283,2022-01-01T02:16:36.000+01:00,126,None,...,False,995.0,True,21.0,7.0,2724.0,4773.0,2791.0,NaN,NaN
1943881,2,meta_clicked_itineraries,vPQ3E34BCYBP9wiWyJP8,1,5009861,2022-01-01T02:18:17.000+01:00,3575610675,2022-01-01T02:17:36.000+01:00,13,None,...,True,320.0,False,0.0,0.0,3902.0,5302.0,3902.0,NaN,NaN
1943882,3,meta_clicked_itineraries,r3s4E34BCwb6y2g4e1fR,1,5009862,2022-01-01T02:19:04.000+01:00,3575611105,2022-01-01T02:18:37.000+01:00,126,None,...,False,890.0,False,0.0,0.0,1638.0,1638.0,1638.0,NaN,NaN
1770649,6535,meta_clicked_itineraries,Yns_E34BCwb6y2g4LGAn,1,5009863,2022-01-01T02:26:22.000+01:00,3575613566,2022-01-01T02:24:14.000+01:00,55,None,...,False,2382.0,True,0.0,0.0,11941.0,40820.0,12066.0,NaN,NaN
1770650,6536,meta_clicked_itineraries,L3tBE34BCwb6y2g4GWbJ,1,5009864,2022-01-01T02:28:28.000+01:00,3575614791,2022-01-01T02:27:18.000+01:00,82,Gpf2uQCTn7fKRC9JSxx6nA,...,False,540.0,False,0.0,0.0,4204.0,4204.0,4204.0,NaN,NaN
1770651,6537,meta_clicked_itineraries,sHtCE34BCwb6y2g4QWfH,1,5009865,2022-01-01T02:29:45.000+01:00,3575615272,2022-01-01T02:28:43.000+01:00,37,wNLn$N8iRaIjT860iyLOsg,...,False,1991.0,False,0.0,0.0,24012.0,54732.0,25240.0,NaN,NaN
1770652,6538,meta_clicked_itineraries,5WlDE34B_h5ol0SVLYR5,1,5009866,2022-01-01T02:30:43.000+01:00,3575615715,2022-01-01T02:29:49.000+01:00,82,yUYZRlEMYZaLSR4TCV0QLg,...,False,645.0,False,0.0,0.0,2732.0,4338.0,3243.0,NaN,NaN
1943892,13,meta_clicked_itineraries,t3tTE34BCwb6y2g4BnsQ,1,5009876,2022-01-01T02:48:04.000+01:00,3575617118,2022-01-01T02:33:06.000+01:00,82,QD2bZFR9MZzWQpEUl47dXQ,...,False,1730.0,False,0.0,0.0,7080.0,11988.0,9248.0,NaN,NaN
1943884,5,meta_clicked_itineraries,GWlKE34B_h5ol0SVBI5o,1,5009868,2022-01-01T02:38:13.000+01:00,3575618248,2022-01-01T02:35:49.000+01:00,55,None,...,False,2335.0,True,272.0,9.0,13788.0,23260.0,14106.0,NaN,NaN


In [63]:
clicks.columns.values

array(['clicks_index', 'clicks_index', 'clicks_id', 'clicks_score',
       'clicks_id', 'clicks_created_at', 'clicks_search_id',
       'clicks_search_created_at', 'clicks_gate_id', 'clicks_meta_ref',
       'clicks_searchrule_id', 'clicks_searchrule_version_id',
       'clicks_pricerule_flight_version_id',
       'clicks_pricerule_hotel_version_id',
       'clicks_pricerule_meta_version_id',
       'clicks_pricerule_ancillary_version_id',
       'clicks_presentationrule_version_id', 'clicks_uuid',
       'clicks_itinerary_string', 'clicks_ip_address', 'clicks_mobile',
       'clicks_searchrule_action_id',
       'clicks_itinerary_marketing_carriers',
       'clicks_itinerary_operating_carriers', 'clicks_itinerary_currency',
       'clicks_itinerary_sales_price', 'clicks_itinerary_sales_price_pax',
       'clicks_itinerary_segment_count', 'clicks_itinerary_direct_flight',
       'clicks_itinerary_travel_time', 'clicks_itinerary_with_baggage',
       'clicks_result_set_position',
      

In [64]:
# Merge the clicks and orders data frames based on the search_id column
clicks_orders_merge_raw = clicks.merge(
    orders, 
    left_on='clicks_search_id', 
    right_on='orders_search_id', 
    suffixes=('_clicks', '_orders')).sort_values(by='clicks_search_id') # Order the merged data frame by the search_id column

In [65]:
clicks_orders_merge_raw.to_csv("data/clicks_orders_merged.csv", index=False)

### Deleting unncessary variables from the environment

In [ ]:
# remove clicks_orders_merge_raw from the environment
del clicks_orders_merge_raw
del clicks
del orders

#### Reading the merged data frame of orders and clicks

In [77]:
clicks_orders_merge = pd.read_csv("data/clicks_orders_merged.csv")

/var/folders/gp/cz1c1p2d6ln8dw3vyy038cwm0000gn/T/ipykernel_72289/343046090.py:1: DtypeWarning: Columns (83,116,123,124,126,129,130,131,132,133) have mixed types. Specify dtype option on import or set low_memory=False.
  clicks_orders_merge = pd.read_csv("data/clicks_orders_merged.csv")


In [78]:
clicks_orders_merge['orders_if_order'] = np.where(clicks_orders_merge['orders_order_id'].isna(),0,1)


### Calculating distances between airports

In [79]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

airports = pd.read_csv("https://raw.githubusercontent.com/ip2location/ip2location-iata-icao/master/iata-icao.csv")
airports = airports[['iata', 'latitude', 'longitude']].rename(columns={'latitude': 'lat', 'longitude': 'lon'})

# adding some missing airports
airports = airports.append({'iata': 'LON', 'lat': airports.loc[airports['iata'] == 'LHR', 'lat'].values[0], 'lon': airports.loc[airports['iata'] == 'LHR', 'lon'].values[0]}, ignore_index=True)
airports = airports.append({'iata': 'PRN', 'lat': 42.6629, 'lon': 21.1655}, ignore_index=True)
airports = airports.append({'iata': 'UBN', 'lat': 47.6514, 'lon': 106.8216}, ignore_index=True)
airports[['lat', 'lon']] = airports[['lat', 'lon']].apply(pd.to_numeric)

print(airports)

def distance_airports(str):
    if pd.isna(str):
        return np.nan
    else:
        origin, destination = str.split('-')
        origin_lon = airports.loc[airports['iata'] == origin, 'lon'].values[0]
        origin_lat = airports.loc[airports['iata'] == origin, 'lat'].values[0]
        destination_lon = airports.loc[airports['iata'] == destination, 'lon'].values[0]
        destination_lat = airports.loc[airports['iata'] == destination, 'lat'].values[0]
        try:
            return round(geodesic((origin_lat, origin_lon), (destination_lat, destination_lon)).km, 2)
        except:
            return None

print(distance_airports('CPH-WAW'))

     iata        lat         lon
0     SHJ  25.328600   55.517200
1     AZI  24.428301   54.458099
2     FJR  25.112200   56.324001
3     XSB  24.283611   52.580278
4     RKT  25.613501   55.938801
...   ...        ...         ...
8980  KAB -16.519800   28.885000
8981  HRE -17.931801   31.092800
8982  LON  51.470600   -0.461941
8983  PRN  42.662900   21.165500
8984  UBN  47.651400  106.821600

[8985 rows x 3 columns]
667.29


/var/folders/gp/cz1c1p2d6ln8dw3vyy038cwm0000gn/T/ipykernel_72289/3821272586.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airports = airports.append({'iata': 'LON', 'lat': airports.loc[airports['iata'] == 'LHR', 'lat'].values[0], 'lon': airports.loc[airports['iata'] == 'LHR', 'lon'].values[0]}, ignore_index=True)
/var/folders/gp/cz1c1p2d6ln8dw3vyy038cwm0000gn/T/ipykernel_72289/3821272586.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airports = airports.append({'iata': 'PRN', 'lat': 42.6629, 'lon': 21.1655}, ignore_index=True)
/var/folders/gp/cz1c1p2d6ln8dw3vyy038cwm0000gn/T/ipykernel_72289/3821272586.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airports = airports.append({'iata': 'UBN', 'lat': 47.6514, 

In [83]:
clicks_orders_merge['clicks_itinerary_string'] = clicks_orders_merge['clicks_itinerary_string'].str.replace(' ','').str.strip()
clicks_orders_merge['clicks_itinerary_string'].apply(lambda x: len(x.split(','))).max()

clicks_itinerary_string_cols = clicks_orders_merge['clicks_itinerary_string'].str.replace(' ','').str.strip().str.split(',').apply(lambda x: pd.Series(x))
clicks_itinerary_string_cols.columns = ['clicks_itinerary_string_' + str(i+1) for i in clicks_itinerary_string_cols.columns]


In [84]:
clicks_itinerary_string_cols.columns.values

array(['clicks_itinerary_string_1', 'clicks_itinerary_string_2',
       'clicks_itinerary_string_3', 'clicks_itinerary_string_4',
       'clicks_itinerary_string_5', 'clicks_itinerary_string_6'],
      dtype=object)

In [86]:

clicks_itinerary_string_cols = clicks_itinerary_string_cols[['clicks_itinerary_string_' + str(i) for i in range(1,7)]]
clicks_itinerary_string_cols[clicks_itinerary_string_cols.isna().any(axis=1)]


,clicks_itinerary_string_1,clicks_itinerary_string_2,clicks_itinerary_string_3,clicks_itinerary_string_4,clicks_itinerary_string_5,clicks_itinerary_string_6
0,CPH-AGP,AGP-CPH,NaN,NaN,NaN,NaN
1,LAX-MCO,MCO-LAX,NaN,NaN,NaN,NaN
2,MUC-ALC,ALC-MUC,NaN,NaN,NaN,NaN
3,MUC-ALC,BCN-MUC,NaN,NaN,NaN,NaN
4,STO-LPA,LPA-STO,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
137422,FRA-ATH,ATH-FRA,NaN,NaN,NaN,NaN
137423,DUS-SFO,LAS-DUS,NaN,NaN,NaN,NaN
137424,CPH-ALC,ALC-CPH,NaN,NaN,NaN,NaN
137425,STR-BGO,BGO-STR,NaN,NaN,NaN,NaN


In [109]:
from itertools import chain


flattened_list = [item for sublist in clicks_itinerary_string_cols.unstack().drop_duplicates().str.split('-').to_list() for item in sublist]
unique_values = set(flattened_list)

print(flattened_list)

# airports['iata'].isin(clicks_itinerary_string_cols.unstack().str.split('-').unstack())


TypeError: 'float' object is not iterable

In [111]:
set(clicks_itinerary_string_cols.unstack().drop_duplicates().str.split('-').to_list())

TypeError: unhashable type: 'list'